In [1]:
import requests,json
import pandas as pd
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import re

In [2]:
def get_all_categories():
    resp = requests.get('https://www.lechocolat-alainducasse.com/uk/')
    cookies = resp.cookies.get_dict()

    headers = {
        'authority': 'www.lechocolat-alainducasse.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9,en-IN;q=0.8',
        'cache-control': 'no-cache',
        'dnt': '1',
        'pragma': 'no-cache',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
    }

    response = requests.get('https://www.lechocolat-alainducasse.com/uk/', cookies=cookies, headers=headers)
    soup = bs(response.content,'lxml')
    categories = list(set([x['href'] for x in soup.find_all('a',{'class':'siteMenuItem__link'})][:-1]))
    return categories

In [3]:
def get_products(collection_link):
    resp = requests.get('https://www.lechocolat-alainducasse.com/uk/')
    cookies = resp.cookies.get_dict()
    headers = {
        'authority': 'www.lechocolat-alainducasse.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9,en-IN;q=0.8',
        'cache-control': 'no-cache',
        'dnt': '1',
        'pragma': 'no-cache',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
    }

    response = requests.get(collection_link, cookies=cookies, headers=headers)
    soup = bs(response.content,'lxml')
    product_links = [x['href'] for x in soup.find('section',{'class':'products__list'}).find_all('a',{'class':'productMiniature__name'})]
    return product_links

In [4]:
def get_product_details(product_link):
    resp = requests.get('https://www.lechocolat-alainducasse.com/uk/')
    cookies = resp.cookies.get_dict()
    headers = {
        'authority': 'www.lechocolat-alainducasse.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9,en-IN;q=0.8',
        'cache-control': 'no-cache',
        'dnt': '1',
        'pragma': 'no-cache',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
    }

    response = requests.get(
        product_link,
        cookies=cookies,
        headers=headers,
    )
    soup = bs(response.content,'lxml')
    for x in soup.find_all('script',{'type':'application/ld+json'}):
        if json.loads(x.text)['@type'] == 'Product':
            break
    inter = json.loads(x.text)
    models = get_models_and_prices(soup,inter)
    details = {}
    details["product_id"] =  soup.find('input',{'id':'product_page_product_id'})['value']
    details["title"] = soup.find("meta",{"property":"og:title"})['content']
    details["image"] = inter["image"]
    details["price"] = soup.find("meta",{"property":"product:price:amount"})["content"]
    details["currency"] = soup.find("meta",{"property":"product:price:currency"})["content"]
    try:
        details["description"] = inter["description"].replace('\xa0',' ')
    except:
        details['description'] = soup.find('meta',{'name':'description'})['content']
    details["sales_prices"] = float(soup.find("meta",{"property":"product:pretax_price:amount"})["content"])
    if models == []:
        details['prices'] = [details['sales_prices'],details['price']]
    else:
        details["prices"] = [m['price'] for m in models]
    details["images"] = [x.split(' ')[0] for x in soup.find('img',{'class':'productImages__image'})['srcset'].split(',')]
    details["url"] = inter["url"]
    try:
        if json.loads(soup.find('script',{'class':'product-data-ga4'}).text)["item_brand"] == '':
            details["brand"] = json.loads(soup.find('script',{'class':'product-data-ga4'}).text)["manufacture"]
        else:
            details["brand"] = json.loads(soup.find('script',{'class':'product-data-ga4'}).text)["item_brand"]
    except:
        if json.loads(soup.find('script',{'class':'single-product-data-ga4'}).text)["item_brand"] == '':
            details["brand"] = json.loads(soup.find('script',{'class':'single-product-data-ga4'}).text)["manufacture"]
        else:
            details["brand"] = json.loads(soup.find('script',{'class':'single-product-data-ga4'}).text)["item_brand"]
    if models == []:
        try:
            wt = soup.find('p',{'class':'productCard__weight'}).text
        except:
            wt = ''
        models = [{"id":soup.find('input',{'id':'product_page_product_id'})['value'],"image":inter["image"],"weight":wt,"price":soup.find("meta",{"property":"product:price:amount"})["content"]}]
    details["models"] = models
    return details

In [5]:
def get_models_and_prices(soup,inter):
    models = []
    resp = requests.get('https://www.lechocolat-alainducasse.com/uk/')
    cookies = resp.cookies.get_dict()
    token = soup.find('input',{'name':'token'})['value']
    product_id = soup.find('input',{'id':'product_page_product_id'})['value']
    soup.find_all('input',{'class':'input-radio'})
    weights = [{'weight':x['title'],'group[6]':x['value']}for x in soup.find_all('input',{'class':'input-radio'})]
    if len(weights) >1:
        for w in weights:
            headers = {
                'authority': 'www.lechocolat-alainducasse.com',
                'accept': 'application/json, text/javascript, */*; q=0.01',
                'accept-language': 'en-US,en;q=0.9,en-IN;q=0.8',
                'cache-control': 'no-cache',
                'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
                'dnt': '1',
                'origin': 'https://www.lechocolat-alainducasse.com',
                'pragma': 'no-cache',
                'referer': 'https://www.lechocolat-alainducasse.com/uk/hazelnut-dragee-dark',
                'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
                'x-requested-with': 'XMLHttpRequest',
            }

            params = {
                'controller': 'product',
                'token': token,
                'id_product': product_id,
                'id_customization': '0',
                'group[6]': w['group[6]'],
                'qty': '1',
            }

            data = {
                'quickview': '0',
                'ajax': '1',
                'action': 'refresh',
                'quantity_wanted': '1',
            }

            response = requests.post(
                'https://www.lechocolat-alainducasse.com/uk/index.php',
                params=params,
                cookies=cookies,
                headers=headers,
                data=data,
            )
            if 'Kg' in w['weight']:
                models.append({"id":w['group[6]'],"weight":w["weight"],"image":inter["image"],"price":float(bs(response.json()['product_prices'],'lxml').find('span').text.replace('£',''))})
            else:
                models.append({"id":w['group[6]'],"weight":w["weight"],"image":inter["image"],"price":float(bs(response.json()['product_prices'],'lxml').find('span').text.replace('£',''))})
    return models

In [6]:
def main():
    product_details = []
    categories = get_all_categories()
    product_links = []
    for c in tqdm(categories,desc='gettign all product links'):
        product_links.extend(get_products(c))
    for p in tqdm(product_links,desc='getting product details'):
        product_details.append(get_product_details(p))
    return product_details

In [8]:
result = main()

getting product details: 100%|████████████████| 137/137 [09:05<00:00,  3.98s/it]


In [9]:
with open('../../output/le_chocolat.json','w') as f:
    json.dump(result,f)